In [86]:
#Librerías Estándar 
import random
import warnings

# Análisis de Datos y Matemáticas 
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
from scipy.interpolate import RegularGridInterpolator
from netCDF4 import Dataset as ncread  # Asumiendo que usas ncread explícitamente

# Visuals
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import animation
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#Utilidades y Entorno (Jupyter/IPython) ---
from tabulate import tabulate
from IPython.display import display, HTML, Image  # Todo en una sola línea
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [113]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import warnings

import sqlite3  # Para trabajar con SQLite
import pandas as pd  # Para manejar datos como DataFrames
import duckdb  # Para trabajar con DuckDB (base de datos en memoria)
import pandas as pd  # Para trabajar con DataFrames
import pandas as pd  # Para manejar datos como DataFrames




warnings.filterwarnings("ignore", category=RuntimeWarning)
# ---------- Paths ----------
path = Path(r"E:/TFG/Datos/Importants/")
data1 = path / "HadISST_sst_v1.1_196001_202105.nc"
data2 = path / "sst.mnmean_v5_196001_202105.nc"
NT = 732  # meses a usar (como en tu notebook)
OUT = path / "_sql_exports"
OUT.mkdir(exist_ok=True)

In [112]:
# ---------- Funciones de carga y procesamiento ----------
def to_lat_lon_latitude_longitude(da: xr.DataArray) -> xr.DataArray:
    """Convierte las coordenadas de latitud y longitud a convenciones estándar."""
    
    # Renombrar "latitude" a "lat" si existe
    if "latitude" in da.coords and "lat" not in da.coords:
        da = da.rename({"latitude": "lat"})
        
    # Renombrar "longitude" a "lon" si existe
    if "longitude" in da.coords and "lon" not in da.coords:
        da = da.rename({"longitude": "lon"})
        
    # Convierte las longitudes a la convención -180...180
    lon = da["lon"]
    lon2 = ((lon + 180) % 360) - 180
    da = da.assign_coords(lon=lon2).sortby("lon")
    
    return da

    


def open_sst_clean(fp: Path, varname: str, nt: int = NT) -> xr.DataArray:
    """Abre el dataset NetCDF y limpia con xarray."""
    ds = xr.open_dataset(fp, decode_cf=True, mask_and_scale=True)
    ds = to_lat_lon_latitude_longitude(ds)
    if varname not in ds:
        raise KeyError(f"{varname} no está en {fp.name}. vars={list(ds.data_vars)}")
    
    
    # Seleccionamos la variable de interés (por ejemplo, "sst") y recortamos el tiempo
    da = ds[varname].isel(time=slice(0, nt))

    # Aplicamos el filtro de valores válidos para SST
    da = da.where((da >= -1.79) & (da <= 35))  # Reemplazar valores fuera del rango con NaN

    # Aseguramos que las longitudes estén en la convención -180 a 180
    return da


# Abre el Dataset completo para cada archivo


def to_float_nan(a):
    """Convierte Variable/MaskedArray a ndarray float con NaNs donde haya máscara."""
    if np.ma.isMaskedArray(a):
        return np.ma.filled(a, np.nan).astype(float)
    return np.array(a, dtype=float)

def calc_monthly_anoms(fen, detrend=True):
    """
    fen: (time, lat, lon) mensual (DataArray)
    ds: Dataset completo (contiene las coordenadas)
    return: anoms (year, month, lat, lon)
    """
    arr = to_float_nan(fen)  # Aseguramos que los valores NaN se convierten a np.nan


    # Si viniera 4D (time, level, lat, lon), toma level 0
    if arr.ndim == 4:
        arr = arr[:, 0, :, :]
    if arr.ndim != 3:
        raise ValueError(f"Se esperaba (time, lat, lon), pero llegó {arr.shape}")

    nt, ny, nx = arr.shape
    nyr = nt // 12
    nt_use = nyr * 12
    dat = arr[:nt_use].copy()

    # Detrend (ajustamos la tendencia temporal si se pide)
    if detrend:
        x = np.arange(nt_use)
        X = np.column_stack([x, np.ones(nt_use)])  # Matriz de diseño (tiempo, 1) para la regresión
        Y = dat.reshape(nt_use, -1)  # Aplanamos el array para usar mínimos cuadrados
        Yf = np.nan_to_num(Y, nan=0.0)  # Convertir NaNs a 0 para la regresión

        coeffs = np.linalg.lstsq(X, Yf, rcond=None)[0]  # Resuelve la regresión lineal
        trend = (X @ coeffs).reshape(dat.shape)  # Calcula la tendencia en el tiempo

        mask = np.isfinite(dat)  # Creamos una máscara de los valores no nulos
        dat[mask] = dat[mask] - trend[mask]  # Restamos la tendencia a los datos no nulos

    # Reorganizar datos a (año, mes, lat, lon)
    dat4 = dat.reshape(nyr, 12, ny, nx)  # (year, month, lat, lon)

    # Calcular la climatología mensual (media de cada mes a lo largo de los años)
    clim = np.nanmean(dat4, axis=0)  # Calculamos la climatología mensual

    # Anomalías: restamos la climatología mensual
    anoms = dat4 - clim[None, :, :, :]  # (year, month, lat, lon)

    # Acceder a las coordenadas lat y lon directamente
    lat = fen.coords["lat"]
    lon = fen.coords["lon"]

    # Devolvemos el DataArray con las anomalías
    return xr.DataArray(
        anoms,
        dims=["year", "month", "lat", "lon"],  # Usamos "lat" y "lon" directamente
        coords={
            "year": np.arange(nyr),
            "month": np.arange(12),
            "lat": lat,  # Usamos las coordenadas de latitud
            "lon": lon,  # Usamos las coordenadas de longitud
        },
    )

def region_mean(anoms: xr.DataArray, region: dict) -> xr.DataArray:
    """Calcula la media de las anomalías para una región específica."""
    # Seleccionar la región usando las coordenadas de latitud y longitud
    anoms_region = anoms.sel(lat=region["lat"], lon=region["lon"])  # Seleccionamos la región

    # Calculamos la media espacial sobre las dimensiones de latitud y longitud
    return anoms_region.mean(dim=["lat", "lon"], skipna=True)  # Media sobre latitudes y longitudes






In [122]:
# Cargar los datos y calcular las anomalías mensuales
had_sst = open_sst_clean(data1, "sst", NT)
ers_sst = open_sst_clean(data2, "sst", NT)

# Seleccionar las regiones Niño3.4 y ATL3 y calcular anomalías
NINO34 = dict(lat=slice(6, -6), lon=slice(-170, -120))
ATL3 = dict(lat=slice(4, -4), lon=slice(-20, 0))

# Calcular las anomalías mensuales para Niño3.4 y ATL3
sst_nino34_anoms = calc_monthly_anoms(had_sst)
sst_atl3_anoms = calc_monthly_anoms(ers_sst)

# Calcular la media por región para Niño 3.4 en ambos datasets
nino34_mean_had = region_mean(sst_nino34_anoms, NINO34)
nino34_mean_ers = region_mean(sst_atl3_anoms, NINO34)

# Calcular la media por región para ATL3 en ambos datasets
atl3_mean_had = region_mean(sst_nino34_anoms, ATL3)
atl3_mean_ers = region_mean(sst_atl3_anoms, ATL3)

# Convertir a DataFrame y añadir etiquetas
nino34_df_had = nino34_mean_had.to_dataframe(name="anomaly").reset_index()
nino34_df_had["region"] = "Nino34"
nino34_df_had["source"] = "HadISST_sst"

nino34_df_ers = nino34_mean_ers.to_dataframe(name="anomaly").reset_index()
nino34_df_ers["region"] = "Nino34"
nino34_df_ers["source"] = "ERSST_sst"

atl3_df_had = atl3_mean_had.to_dataframe(name="anomaly").reset_index()
atl3_df_had["region"] = "ATL3"
atl3_df_had["source"] = "HadISST_sst"

atl3_df_ers = atl3_mean_ers.to_dataframe(name="anomaly").reset_index()
atl3_df_ers["region"] = "ATL3"
atl3_df_ers["source"] = "ERSST_sst"

# Unir todas las tablas
final_table = pd.concat([nino34_df_had, nino34_df_ers, atl3_df_had, atl3_df_ers], ignore_index=True)
# Asegurarse que las fechas estén correctas
final_table["year"] = final_table["year"] + 1960  # Añadir el año base para cada observación
final_table["month"] = final_table["month"] + 1  # Asignar correctamente el mes
print(final_table.head(40))
# Revisa nuevamente el resultado de la consulta
print("Verificación de NaNs en las anomalías:")
print(final_table.isnull().sum())  # Esto te ayudará a ver si los NaNs persisten


# Guardar la tabla como Parquet
regions_parquet = OUT / "regions_timeseries_anomalies_detrended.parquet"
final_table.to_parquet(regions_parquet, index=False)

print("OK ->", regions_parquet)



    year  month   anomaly  region       source
0   1960      1  0.121873  Nino34  HadISST_sst
1   1960      2 -0.139709  Nino34  HadISST_sst
2   1960      3  0.051061  Nino34  HadISST_sst
3   1960      4  0.143723  Nino34  HadISST_sst
4   1960      5  0.209010  Nino34  HadISST_sst
5   1960      6 -0.082081  Nino34  HadISST_sst
6   1960      7 -0.109685  Nino34  HadISST_sst
7   1960      8  0.114966  Nino34  HadISST_sst
8   1960      9  0.177948  Nino34  HadISST_sst
9   1960     10  0.042939  Nino34  HadISST_sst
10  1960     11 -0.268896  Nino34  HadISST_sst
11  1960     12  0.115227  Nino34  HadISST_sst
12  1961      1 -0.019849  Nino34  HadISST_sst
13  1961      2  0.140307  Nino34  HadISST_sst
14  1961      3  0.041600  Nino34  HadISST_sst
15  1961      4  0.227473  Nino34  HadISST_sst
16  1961      5  0.118074  Nino34  HadISST_sst
17  1961      6  0.184192  Nino34  HadISST_sst
18  1961      7 -0.205068  Nino34  HadISST_sst
19  1961      8 -0.153542  Nino34  HadISST_sst
20  1961     

In [123]:
con = duckdb.connect()
# Cargar la tabla Parquet en DuckDB
parquet_file = OUT / "regions_timeseries_anomalies_detrended.parquet"

# Leer la tabla Parquet y cargarla en DuckDB
con.execute(f"CREATE OR REPLACE TABLE regions AS SELECT * FROM read_parquet('{parquet_file}')")

# Hacer una consulta SQL para ver las primeras filas
result = con.execute("SELECT * FROM regions LIMIT 5").fetchall()
print(result)

[(1960, 1, 0.12187341273642381, 'Nino34', 'HadISST_sst'), (1960, 2, -0.13970939875466173, 'Nino34', 'HadISST_sst'), (1960, 3, 0.051060818432104436, 'Nino34', 'HadISST_sst'), (1960, 4, 0.14372293118404975, 'Nino34', 'HadISST_sst'), (1960, 5, 0.20901033672964373, 'Nino34', 'HadISST_sst')]
